# GRPO applications on Gemma3 models (Kaggle Notebook)
In this notebooks, you will find the different GRPO fine-tuning application I discussed in my youtube video.

This notebook contain the code for all the experiments I made.\
You just need to run the code under the section you're interested in, as well as the preambule code to install dependencies and set API keys for Hugging Face and W&B.

**Notebook Structure:**
1. Gemma3 1b
    1. AI detection with ModernBERT
    2. AI detection with DesklibAI
    3. AI detection with DesklibAI + Reasoning
2. Gemma3 4b
    1. AI detection with ModertBERT
    2. AI detection with DesklibAI
    3. AI detection with DesklibAI + Reasoning
    4. Sentiment (ClapAI) + Reasoning
    5. Sentiment (ClapAI)
    6. Sentiment (ClapAI) + Binary reward


<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://www.kaggle.com/"><img src="https://upload.wikimedia.org/wikipedia/commons/7/7c/Kaggle_logo.png" width="100"></a>
</div>

## Installation

#### Training with DesklibAI reward model
Using DeskLibAI model in a reward function produces an error with the current version of unsloth-zoo. Install it instead from my fork.

In [ ]:
%%capture
!pip install --force-reinstall --no-cache-dir "unsloth_zoo @ git+https://github.com/HmzBo/unsloth-zoo.git@fix-encoding"

In [ ]:
%%capture
!pip install unsloth==2025.3.18 vllm
!pip install trl==0.16.0
!pip install triton==3.1.0
!pip install -U pynvml
!pip install transformers==4.50.1 bitsandbytes peft==0.14.0 cut_cross_entropy
!pip install --no-deps accelerate xformers==0.0.29.post3
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

#### Training without DesklibAI reward model

In [ ]:
 %%capture
!pip install unsloth==2025.3.18 vllm
!pip install trl==0.16.0
!pip install triton==3.1.0
!pip install -U pynvml
!pip install transformers==4.50.1 bitsandbytes peft==0.14.0 cut_cross_entropy unsloth_zoo==2025.3.16
!pip install --no-deps accelerate xformers==0.0.29.post3
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

#### Setup W&B

In [ ]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_key = user_secrets.get_secret("wandb")

wandb.login(key=wandb_key)
wandb.init(project="kaggle", name="gemma-3-4b_detect_grpo_unsloth")

#### HF Login

In [ ]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

# Retrieve token from Kaggle secrets
token = user_secrets.get_secret("hf_token")
login(token)

## Gemma3 1B Experiments

### Task: AI text detector with ModernBERT model

#### Load Model and Add LoRA

In [ ]:
from unsloth import FastModel
import torch

max_seq_length = 1024

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
)

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 32,           
    lora_alpha = 32,  
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

#### Data Preparation

In [ ]:
from datasets import load_dataset, Dataset

SYSTEM_PROMPT = f"""
You are a helpful and conversational assistant. Respond in a human-like style.
"""

# load dataset function
def get_instructions() -> Dataset:
    # Load the dataset
    dataset_id = "dmitva/human_ai_generated_text"
    raw_dataset = load_dataset(dataset_id, split="train[:10%]")
    data = raw_dataset.map(lambda x: {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['instructions'].split("Task: ")[1]}
        ]
    },
    remove_columns=['id', 'human_text', 'ai_text', 'instructions']
    )
    return data

dataset = get_instructions()

#### Load ModernBERT model

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import PeftModel

# Load the base ModernBERT model
model_id = "answerdotai/ModernBERT-base"
base_model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=2,
    device_map={"":1}  # Force to cuda:1
)

# Load LoRA adapters and merge into base model
peft_model_id = "HmzBou/modernbert-lora-ai-detection"
mbert_model = PeftModel.from_pretrained(base_model, peft_model_id, device_map={"":1})
mbert_model = mbert_model.merge_and_unload()

# Load tokenizer and set max_length
mbert_tokenizer = AutoTokenizer.from_pretrained(model_id)
mbert_tokenizer.model_max_length = 1024

In [ ]:
import torch

# Direct model inference function
def model_predict(text):
    inputs = mbert_tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=1024,
        return_tensors="pt"
    ).to("cuda:1")

    with torch.no_grad():
        outputs = mbert_model(**inputs)

    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return {
        "human_prob": probs[0][0].item(),
        "ai_prob": probs[0][1].item()
    }

#### Reward function 

In [ ]:
# Custom reward function for GRPOTrainer
def custom_reward_function(prompts, completions, **kwargs):

    rewards = []
    responses = [completion[0]["content"] for completion in completions]
    print("="*50)
    last_prompt = ""
    for prompt, response in zip(prompts, responses):
        prediction = model_predict(response)
        reward = prediction["human_prob"] - prediction["ai_prob"]
        rewards.append(reward)
        if last_prompt != prompt:
            print(f"Prompt:\n{prompt[1]['content']}")
            print("*"*20)
        print(f"Response:\n{response[:200]}{'...' if len(response) > 200 else ''} Reward: {reward}")
        print("-"*20)
        last_prompt = prompt
    return rewards

#### Model Training

In [ ]:
from trl import GRPOConfig, GRPOTrainer
from unsloth import is_bfloat16_supported

max_prompt_length = 256

training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 2, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    max_steps = 200,
    save_steps = 200,
    max_grad_norm = 0.1,
    report_to = "wandb", # Can use Weights & Biases
    output_dir = "outputs",
)

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        custom_reward_function
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

### Task: AI text detector with DesklibAI model

#### Load Model and Add LoRA

In [ ]:
from unsloth import FastModel
import torch

max_seq_length = 1024

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 64,           
    lora_alpha = 64,  
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

#### Data Preparation

In [ ]:
from datasets import load_dataset, Dataset

SYSTEM_PROMPT = f"""
You are a helpful and conversational assistant. Respond in a human-like style.
"""

# load dataset function
def get_instructions() -> Dataset:
    # Load the dataset
    dataset_id = "dmitva/human_ai_generated_text"
    raw_dataset = load_dataset(dataset_id, split="train[:10%]")
    data = raw_dataset.map(lambda x: {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['instructions'].split("Task: ")[1]}
        ]
    },
    remove_columns=['id', 'human_text', 'ai_text', 'instructions']
    )
    return data

dataset = get_instructions()

#### Load DesklibAI model

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, AutoModel, PreTrainedModel

class DesklibAIDetectionModel(PreTrainedModel):
    config_class = AutoConfig

    def __init__(self, config):
        super().__init__(config)
        # Initialize the base transformer model.
        self.model = AutoModel.from_config(config)
        # Define a classifier head.
        self.classifier = nn.Linear(config.hidden_size, 1)
        # Initialize weights (handled by PreTrainedModel)
        self.init_weights()

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Forward pass through the transformer
        outputs = self.model(input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs[0]
        # Mean pooling
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, dim=1)
        sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
        pooled_output = sum_embeddings / sum_mask

        # Classifier
        logits = self.classifier(pooled_output)
        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits.view(-1), labels.float())

        output = {"logits": logits}
        if loss is not None:
            output["loss"] = loss
        return output

def predict_single_text(text, model, tokenizer, device, max_len=768, threshold=0.5):
    encoded = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors='pt'
    )
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs["logits"]
        probability = torch.sigmoid(logits).item()

    label = 1 if probability >= threshold else 0
    return probability, label

In [ ]:
model_directory = "desklib/ai-text-detector-v1.01"

desklibai_tokenizer = AutoTokenizer.from_pretrained(model_directory)
desklibai_model = DesklibAIDetectionModel.from_pretrained(model_directory)

desklibai_model.to("cuda:1")

#### Reward function

In [ ]:
# Custom reward function for GRPOTrainer
def custom_reward_function(prompts, completions, **kwargs):
    rewards = []
    responses = [completion[0]["content"] for completion in completions]
    print("="*50)
    last_prompt = ""
    for prompt, response in zip(prompts, responses):
        probability, predicted_label = predict_single_text(response, desklibai_model, desklibai_tokenizer, "cuda:1")
        reward = (1 - probability) - probability
        rewards.append(reward)
        if last_prompt != prompt:
            print(f"Prompt:\n{prompt[1]['content']}")
            print("*"*20)
        print(f"Response:\n{response[:200]}{'...' if len(response) > 200 else ''} Reward: {reward}")
        print("-"*20)
        last_prompt = prompt
    return rewards

#### Model training

In [ ]:
from trl import GRPOConfig, GRPOTrainer
from unsloth import is_bfloat16_supported

max_prompt_length = 256

training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 2, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    max_steps = 200,
    save_steps = 200,
    max_grad_norm = 0.1,
    report_to = "wandb", # Can use Weights & Biases
    output_dir = "outputs",
)

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        custom_reward_function
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

### Task: AI text detector with DesklibAI model + Reasoning

#### Load Model and Add LoRA

In [ ]:
from unsloth import FastModel
import torch

max_seq_length = 1024

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 64,           
    lora_alpha = 64,  
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

#### Data Preparation

In [ ]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
reasoning_start = "<thinking>"
reasoning_end   = "</thinking>"
response_start = "<human_like_response>"
response_end = "</human_like_response>"


SYSTEM_PROMPT = f"""
You are a helpful and conversational assistant.
Place your thinking between {reasoning_start} and {reasoning_end} tags.
Then, provide your human-like response between {response_start} and {response_end} tags.
"""

# load dataset function
def get_instructions() -> Dataset:
    # Load the dataset
    dataset_id = "dmitva/human_ai_generated_text"
    raw_dataset = load_dataset(dataset_id, split="train[:10%]")
    data = raw_dataset.map(lambda x: {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['instructions'].split("Task: ")[1]}
        ]
    },
    remove_columns=['id', 'human_text', 'ai_text', 'instructions']
    )
    return data

dataset = get_instructions()

#### Load DesklibAI model

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, AutoModel, PreTrainedModel

class DesklibAIDetectionModel(PreTrainedModel):
    config_class = AutoConfig

    def __init__(self, config):
        super().__init__(config)
        # Initialize the base transformer model.
        self.model = AutoModel.from_config(config)
        # Define a classifier head.
        self.classifier = nn.Linear(config.hidden_size, 1)
        # Initialize weights (handled by PreTrainedModel)
        self.init_weights()

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Forward pass through the transformer
        outputs = self.model(input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs[0]
        # Mean pooling
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, dim=1)
        sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
        pooled_output = sum_embeddings / sum_mask

        # Classifier
        logits = self.classifier(pooled_output)
        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits.view(-1), labels.float())

        output = {"logits": logits}
        if loss is not None:
            output["loss"] = loss
        return output

def predict_single_text(text, model, tokenizer, device, max_len=768, threshold=0.5):
    encoded = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors='pt'
    )
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs["logits"]
        probability = torch.sigmoid(logits).item()

    label = 1 if probability >= threshold else 0
    return probability, label

In [ ]:
model_directory = "desklib/ai-text-detector-v1.01"

desklibai_tokenizer = AutoTokenizer.from_pretrained(model_directory)
desklibai_model = DesklibAIDetectionModel.from_pretrained(model_directory)

desklibai_model.to("cuda:1")

#### Reward functions

In [ ]:
# Custom reward function for GRPOTrainer
def custom_reward_function(prompts, completions, **kwargs):
    rewards = []
    responses = [completion[0]["content"] for completion in completions]
    print("="*50)
    last_prompt = ""
    for prompt, response in zip(prompts, responses):
        match_resp = re.search(r"<human_like_response>(.*?)</human_like_response>", response)
        match_think = re.search(r"<thinking>(.*?)</thinking>", response)
        if match_resp:
            response = match_resp.group(1)
            probability, predicted_label = predict_single_text(response, desklibai_model, desklibai_tokenizer, "cuda:1")
            reward = (1 - probability) - probability
        else:
            reward = -1
        rewards.append(reward)
        if last_prompt != prompt:
            print(f"Prompt:\n{prompt[1]['content']}")
            print("*"*20)
        if match_think:
            thinking_trace = match_think.group(1)
            print(f"Thinking:\n{thinking_trace[:200]}{'...' if len(thinking_trace) > 200 else ''}")
        if match_resp:
            print(f"Response:\n{response[:200]}{'...' if len(response) > 200 else ''} Reward: {reward}")
        print("-"*20)
        last_prompt = prompt

    return rewards

In [ ]:
import re

match_format = re.compile(
    rf"^[\s]{{0,}}"\
    rf"{reasoning_start}.+?{reasoning_end}.*?"\
    rf"{response_start}(.+?){response_end}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)

In [ ]:
def match_format_exactly(completions, **kwargs):
    rewards = []
    for completion in completions:
        reward = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: reward += 1.0
        rewards.append(reward)
    return rewards

In [ ]:
def match_format_approximately(completions, **kwargs):
    rewards = []
    for completion in completions:
        reward = 0
        response = completion[0]["content"]
        # Count how many keywords are seen - we penalize if too many!
        # If we see 1, then plus some points!
        reward += 0.5 if response.count(reasoning_start) == 1 else -0.5
        reward += 0.5 if response.count(reasoning_end)   == 1 else -0.5
        reward += 0.5 if response.count(response_start)  == 1 else -0.5
        reward += 0.5 if response.count(response_end)    == 1 else -0.5
        rewards.append(reward)
    return rewards

#### Model training

In [ ]:
from trl import GRPOConfig, GRPOTrainer
from unsloth import is_bfloat16_supported

max_prompt_length = 256

training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 2, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    max_steps = 200,
    save_steps = 200,
    max_grad_norm = 0.1,
    report_to = "wandb", # Can use Weights & Biases
    output_dir = "outputs",
)

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        custom_reward_function,
        match_format_exactly,
        match_format_approximately
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

## Gemma3 4B Experiments

### Task: AI text detector with ModernBERT model

#### Load Model and Add LoRA

In [ ]:
from unsloth import FastModel
import torch

max_seq_length = 1024

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 64,           
    lora_alpha = 64,  
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

#### Data Preparation

In [ ]:
from datasets import load_dataset, Dataset

SYSTEM_PROMPT = f"""
You are a helpful and conversational assistant. Respond in a human-like style.
"""

# load dataset function
def get_instructions() -> Dataset:
    # Load the dataset
    dataset_id = "dmitva/human_ai_generated_text"
    raw_dataset = load_dataset(dataset_id, split="train[:10%]")
    data = raw_dataset.map(lambda x: {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['instructions'].split("Task: ")[1]}
        ]
    },
    remove_columns=['id', 'human_text', 'ai_text', 'instructions']
    )
    return data

dataset = get_instructions()

#### Load ModernBERT model

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import PeftModel

# Load the base ModernBERT model
model_id = "answerdotai/ModernBERT-base"
base_model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=2,
    device_map={"":1}  # Force to cuda:1
)

# Load LoRA adapters and merge into base model
peft_model_id = "HmzBou/modernbert-lora-ai-detection"
mbert_model = PeftModel.from_pretrained(base_model, peft_model_id, device_map={"":1})
mbert_model = mbert_model.merge_and_unload()

# Load tokenizer and set max_length
mbert_tokenizer = AutoTokenizer.from_pretrained(model_id)
mbert_tokenizer.model_max_length = 1024

In [ ]:
import torch

# Direct model inference function
def model_predict(text):
    inputs = mbert_tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=1024,
        return_tensors="pt"
    ).to("cuda:1")

    with torch.no_grad():
        outputs = mbert_model(**inputs)

    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return {
        "human_prob": probs[0][0].item(),
        "ai_prob": probs[0][1].item()
    }

#### Reward function 

In [ ]:
# Custom reward function for GRPOTrainer
def custom_reward_function(prompts, completions, **kwargs):

    rewards = []
    responses = [completion[0]["content"] for completion in completions]
    print("="*50)
    last_prompt = ""
    for prompt, response in zip(prompts, responses):
        prediction = model_predict(response)
        reward = 4*prediction["human_prob"] - prediction["ai_prob"]
        rewards.append(reward)
        if last_prompt != prompt:
            print(f"Prompt:\n{prompt[1]['content']}")
            print("*"*20)
        print(f"Response:\n{response[:200]}{'...' if len(response) > 200 else ''} Reward: {reward}")
        print("-"*20)
        last_prompt = prompt
    return rewards

#### Model Training

In [ ]:
from trl import GRPOConfig, GRPOTrainer
from unsloth import is_bfloat16_supported

max_prompt_length = 256

training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 2, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    max_steps = 200,
    save_steps = 200,
    max_grad_norm = 0.1,
    report_to = "wandb", # Can use Weights & Biases
    output_dir = "outputs",
)

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        custom_reward_function
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

### Task: AI text detector with DesklibAI model

#### Load Model and Add LoRA

In [ ]:
from unsloth import FastModel
import torch

max_seq_length = 1024

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 64,           
    lora_alpha = 64,  
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

#### Data Preparation

In [ ]:
from datasets import load_dataset, Dataset

SYSTEM_PROMPT = f"""
You are a helpful and conversational assistant. Respond in a human-like style.
"""

# load dataset function
def get_instructions() -> Dataset:
    # Load the dataset
    dataset_id = "dmitva/human_ai_generated_text"
    raw_dataset = load_dataset(dataset_id, split="train[:10%]")
    data = raw_dataset.map(lambda x: {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['instructions'].split("Task: ")[1]}
        ]
    },
    remove_columns=['id', 'human_text', 'ai_text', 'instructions']
    )
    return data

dataset = get_instructions()

#### Load DesklibAI model

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, AutoModel, PreTrainedModel

class DesklibAIDetectionModel(PreTrainedModel):
    config_class = AutoConfig

    def __init__(self, config):
        super().__init__(config)
        # Initialize the base transformer model.
        self.model = AutoModel.from_config(config)
        # Define a classifier head.
        self.classifier = nn.Linear(config.hidden_size, 1)
        # Initialize weights (handled by PreTrainedModel)
        self.init_weights()

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Forward pass through the transformer
        outputs = self.model(input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs[0]
        # Mean pooling
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, dim=1)
        sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
        pooled_output = sum_embeddings / sum_mask

        # Classifier
        logits = self.classifier(pooled_output)
        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits.view(-1), labels.float())

        output = {"logits": logits}
        if loss is not None:
            output["loss"] = loss
        return output

def predict_single_text(text, model, tokenizer, device, max_len=768, threshold=0.5):
    encoded = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors='pt'
    )
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs["logits"]
        probability = torch.sigmoid(logits).item()

    label = 1 if probability >= threshold else 0
    return probability, label

In [ ]:
model_directory = "desklib/ai-text-detector-v1.01"

desklibai_tokenizer = AutoTokenizer.from_pretrained(model_directory)
desklibai_model = DesklibAIDetectionModel.from_pretrained(model_directory)

desklibai_model.to("cuda:1")

#### Reward function

In [ ]:
# Custom reward function for GRPOTrainer
def custom_reward_function(prompts, completions, **kwargs):
    rewards = []
    responses = [completion[0]["content"] for completion in completions]
    print("="*50)
    last_prompt = ""
    for prompt, response in zip(prompts, responses):
        probability, predicted_label = predict_single_text(response, desklibai_model, desklibai_tokenizer, "cuda:1")
        reward = 4*(1 - probability) - probability
        rewards.append(reward)
        if last_prompt != prompt:
            print(f"Prompt:\n{prompt[1]['content']}")
            print("*"*20)
        print(f"Response:\n{response[:200]}{'...' if len(response) > 200 else ''} Reward: {reward}")
        print("-"*20)
        last_prompt = prompt
    return rewards

#### Model training

In [ ]:
from trl import GRPOConfig, GRPOTrainer
from unsloth import is_bfloat16_supported

max_prompt_length = 256

training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 2, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    max_steps = 200,
    save_steps = 200,
    max_grad_norm = 0.1,
    report_to = "wandb", # Can use Weights & Biases
    output_dir = "outputs",
)

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        custom_reward_function
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

### Task: AI text detector with DesklibAI model + Reasoning

#### Load Model and Add LoRA

In [ ]:
from unsloth import FastModel
import torch

max_seq_length = 1024

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 64,           
    lora_alpha = 64,  
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

#### Data Preparation

In [ ]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
reasoning_start = "<thinking>"
reasoning_end   = "</thinking>"
response_start = "<human_like_response>"
response_end = "</human_like_response>"
# Better response tags
response_start = "<RESPONSE>"
response_end = "</RESPONSE>"


SYSTEM_PROMPT = f"""
You are a helpful and conversational assistant.
Place your thinking between {reasoning_start} and {reasoning_end} tags.
Then, provide your human-like response between {response_start} and {response_end} tags.
"""

# Better system prompts
SYSTEM_PROMPT = f"""
You are a helpful and conversational assistant. Think about the instruction and provide your thinking process, then you need to respond in a human-like writing style.
Place your thinking between {reasoning_start} and {reasoning_end} tags.
Then, provide your human-like response between {response_start} and {response_end} tags.
"""


# load dataset function
def get_instructions() -> Dataset:
    # Load the dataset
    dataset_id = "dmitva/human_ai_generated_text"
    raw_dataset = load_dataset(dataset_id, split="train[:10%]")
    data = raw_dataset.map(lambda x: {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['instructions'].split("Task: ")[1]}
        ]
    },
    remove_columns=['id', 'human_text', 'ai_text', 'instructions']
    )
    return data

dataset = get_instructions()

#### Load DesklibAI model

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, AutoModel, PreTrainedModel

class DesklibAIDetectionModel(PreTrainedModel):
    config_class = AutoConfig

    def __init__(self, config):
        super().__init__(config)
        # Initialize the base transformer model.
        self.model = AutoModel.from_config(config)
        # Define a classifier head.
        self.classifier = nn.Linear(config.hidden_size, 1)
        # Initialize weights (handled by PreTrainedModel)
        self.init_weights()

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Forward pass through the transformer
        outputs = self.model(input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs[0]
        # Mean pooling
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, dim=1)
        sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
        pooled_output = sum_embeddings / sum_mask

        # Classifier
        logits = self.classifier(pooled_output)
        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits.view(-1), labels.float())

        output = {"logits": logits}
        if loss is not None:
            output["loss"] = loss
        return output

def predict_single_text(text, model, tokenizer, device, max_len=768, threshold=0.5):
    encoded = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors='pt'
    )
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs["logits"]
        probability = torch.sigmoid(logits).item()

    label = 1 if probability >= threshold else 0
    return probability, label

In [ ]:
model_directory = "desklib/ai-text-detector-v1.01"

desklibai_tokenizer = AutoTokenizer.from_pretrained(model_directory)
desklibai_model = DesklibAIDetectionModel.from_pretrained(model_directory)

desklibai_model.to("cuda:1")

#### Reward functions

In [ ]:
# Custom reward function for GRPOTrainer
def custom_reward_function(prompts, completions, **kwargs):
    rewards = []
    responses = [completion[0]["content"] for completion in completions]
    print("="*50)
    last_prompt = ""
    for prompt, response in zip(prompts, responses):
        match_resp = re.search(r"<human_like_response>(.*?)</human_like_response>", response)
        match_think = re.search(r"<thinking>(.*?)</thinking>", response)
        if match_resp:
            response = match_resp.group(1)
            probability, predicted_label = predict_single_text(response, desklibai_model, desklibai_tokenizer, "cuda:1")
            reward = 4*(1 - probability) - probability
        else:
            reward = -1
        rewards.append(reward)
        if last_prompt != prompt:
            print(f"Prompt:\n{prompt[1]['content']}")
            print("*"*20)
        if match_think:
            thinking_trace = match_think.group(1)
            print(f"Thinking:\n{thinking_trace[:200]}{'...' if len(thinking_trace) > 200 else ''}")
        if match_resp:
            print(f"Response:\n{response[:200]}{'...' if len(response) > 200 else ''} Reward: {reward}")
        print("-"*20)
        last_prompt = prompt

    return rewards

In [ ]:
import re

match_format = re.compile(
    rf"^[\s]{{0,}}"\
    rf"{reasoning_start}.+?{reasoning_end}.*?"\
    rf"{response_start}(.+?){response_end}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)

In [ ]:
def match_format_exactly(completions, **kwargs):
    rewards = []
    for completion in completions:
        reward = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: reward += 1.0
        rewards.append(reward)
    return rewards

In [ ]:
def match_format_approximately(completions, **kwargs):
    rewards = []
    for completion in completions:
        reward = 0
        response = completion[0]["content"]
        # Count how many keywords are seen - we penalize if too many!
        # If we see 1, then plus some points!
        reward += 0.5 if response.count(reasoning_start) == 1 else -0.5
        reward += 0.5 if response.count(reasoning_end)   == 1 else -0.5
        reward += 0.5 if response.count(response_start)  == 1 else -0.5
        reward += 0.5 if response.count(response_end)    == 1 else -0.5
        rewards.append(reward)
    return rewards

#### Model training

In [ ]:
from trl import GRPOConfig, GRPOTrainer
from unsloth import is_bfloat16_supported

max_prompt_length = 256

training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 2, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    max_steps = 200,
    save_steps = 200,
    max_grad_norm = 0.1,
    report_to = "wandb", # Can use Weights & Biases
    output_dir = "outputs",
)

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        custom_reward_function,
        match_format_exactly,
        match_format_approximately
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

### Task: Negative Sentiment with ClapAI model + Reasoning

#### Load Model and Add LoRA

In [ ]:
from unsloth import FastModel
import torch

max_seq_length = 1024

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 64,           
    lora_alpha = 64,  
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

#### Data Preparation

In [ ]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
reasoning_start = "<thinking>"
reasoning_end   = "</thinking>"
response_start = "<neg_sent_response>"
response_end = "</neg_sent_response>"


SYSTEM_PROMPT = f"""
You are a helpful and conversational assistant.
When responding, first analyze the user's instruction carefully and think through your reasoning for why a negative sentiment is appropriate.
Enclose your reasoning between the tags {reasoning_start} and {reasoning_end}.
Afterward, provide your response with a clearly negative sentiment, enclosed between the tags {response_start} and {response_end}.
Always follow this format and ensure your response reflects negative sentiment.
"""

# load dataset function
def get_instructions() -> Dataset:
    # Load the dataset
    dataset_id = "dmitva/human_ai_generated_text"
    raw_dataset = load_dataset(dataset_id, split="train[:10%]")
    data = raw_dataset.map(lambda x: {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['instructions'].split("Task: ")[1]}
        ]
    },
    remove_columns=['id', 'human_text', 'ai_text', 'instructions']
    )
    return data

dataset = get_instructions()

#### Load ClapAI model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_id = "clapAI/modernBERT-base-multilingual-sentiment"
# Load the tokenizer and model
tokenizer_sent = AutoTokenizer.from_pretrained(model_id)
model_sent = AutoModelForSequenceClassification.from_pretrained(model_id, torch_dtype=torch.float16)

model_sent.to("cuda:1")
model_sent.eval()

# Retrieve labels from the model's configuration
id2label = model_sent.config.id2label

#### Reward functions

In [ ]:
# Custom reward function for GRPOTrainer
def custom_reward_function(prompts, completions, **kwargs):
    rewards = []
    responses = [completion[0]["content"] for completion in completions]
    print("="*50)
    last_prompt = ""
    for prompt, response in zip(prompts, responses):
        match_resp = re.search(r"<neg_sent_response>(.*?)</neg_sent_response>", response)
        match_think = re.search(r"<thinking>(.*?)</thinking>", response)
        if match_resp:
            response = match_resp.group(1)

        length = len(response)
        # Length thresholds
        min_len = 10     # penalize if shorter
        max_len = 2000    # penalize if longer

        if length < min_len or length > max_len:
            reward = 0
        else:
            reward = -((length - 10) * (length - 2000)) / 1_000_000
            inputs = tokenizer_sent(response, return_tensors="pt").to("cuda:1")
            with torch.inference_mode():
                outputs = model_sent(**inputs)
                predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
                neg = predictions[0][0]
            reward += 4 * neg - (1 - neg)
        rewards.append(reward)
        if last_prompt != prompt:
            print(f"Prompt:\n{prompt[1]['content']}")
            print("*"*20)
        if match_think:
            thinking_trace = match_think.group(1)
            print(f"Thinking:\n{thinking_trace[:200]}{'...' if len(thinking_trace) > 200 else ''}")
        if match_resp:
            print(f"Response:\n{response[:200]}{'...' if len(response) > 200 else ''} Reward: {reward}")
        print("-"*20)
        last_prompt = prompt

    return rewards

In [ ]:
import re

match_format = re.compile(
    rf"^[\s]{{0,}}"\
    rf"{reasoning_start}.+?{reasoning_end}.*?"\
    rf"{response_start}(.+?){response_end}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)

In [ ]:
def match_format_exactly(completions, **kwargs):
    rewards = []
    for completion in completions:
        reward = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: reward += 1.0
        rewards.append(reward)
    return rewards

In [ ]:
def match_format_approximately(completions, **kwargs):
    rewards = []
    for completion in completions:
        reward = 0
        response = completion[0]["content"]
        # Count how many keywords are seen - we penalize if too many!
        # If we see 1, then plus some points!
        reward += 0.5 if response.count(reasoning_start) == 1 else -0.5
        reward += 0.5 if response.count(reasoning_end)   == 1 else -0.5
        reward += 0.5 if response.count(response_start)  == 1 else -0.5
        reward += 0.5 if response.count(response_end)    == 1 else -0.5
        rewards.append(reward)
    return rewards

#### Model training

In [ ]:
from trl import GRPOConfig, GRPOTrainer
from unsloth import is_bfloat16_supported

max_prompt_length = 256

training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 1, 
    num_generations = 2,
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    max_steps = 200,
    save_steps = 200,
    max_grad_norm = 0.1,
    report_to = "wandb", # Can use Weights & Biases
    output_dir = "outputs",
)

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        custom_reward_function,
        match_format_exactly,
        match_format_approximately
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

### Task: Negative Sentiment with ClapAI model

#### Load Model and Add LoRA

In [ ]:
from unsloth import FastModel
import torch

max_seq_length = 1024

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 64,           
    lora_alpha = 64,  
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

#### Data Preparation

In [ ]:
import re
from datasets import load_dataset, Dataset


SYSTEM_PROMPT = f"""
You are a helpful and conversational assistant. Always respond with a clearly negative sentiment.
"""

# load dataset function
def get_instructions() -> Dataset:
    # Load the dataset
    dataset_id = "dmitva/human_ai_generated_text"
    raw_dataset = load_dataset(dataset_id, split="train[:10%]")
    data = raw_dataset.map(lambda x: {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['instructions'].split("Task: ")[1]}
        ]
    },
    remove_columns=['id', 'human_text', 'ai_text', 'instructions']
    )
    return data

dataset = get_instructions()

#### Load ClapAI model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_id = "clapAI/modernBERT-base-multilingual-sentiment"
# Load the tokenizer and model
tokenizer_sent = AutoTokenizer.from_pretrained(model_id)
model_sent = AutoModelForSequenceClassification.from_pretrained(model_id, torch_dtype=torch.float16)

model_sent.to("cuda:1")
model_sent.eval()

# Retrieve labels from the model's configuration
id2label = model_sent.config.id2label

#### Reward functions

In [ ]:
# Custom reward function for GRPOTrainer
def custom_reward_function(prompts, completions, **kwargs):
    rewards = []
    responses = [completion[0]["content"] for completion in completions]
    print("="*50)
    last_prompt = ""
    for prompt, response in zip(prompts, responses):

        length = len(response)

        # Length thresholds
        min_len = 10     # penalize if shorter
        max_len = 2000    # penalize if longer

        if length < min_len or length > max_len:
            reward = 0
        else:
            reward = -((length - 10) * (length - 2000)) / 1_000_000
            inputs = tokenizer_sent(response, return_tensors="pt").to("cuda:1")
            with torch.inference_mode():
                outputs = model_sent(**inputs)
                predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
                neg = predictions[0][0]
            reward += 4 * neg - (1 - neg)
        rewards.append(reward)
        if last_prompt != prompt:
            print(f"Prompt:\n{prompt[1]['content']}")
            print("*"*20)
        print(f"Response:\n{response[:200]}{'...' if len(response) > 200 else ''} Reward: {reward}")
        print("-"*20)
        last_prompt = prompt

    return rewards

#### Model training

In [ ]:
from trl import GRPOConfig, GRPOTrainer
from unsloth import is_bfloat16_supported

max_prompt_length = 256

training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 1, 
    num_generations = 2,
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    max_steps = 200,
    save_steps = 200,
    max_grad_norm = 0.1,
    report_to = "wandb", # Can use Weights & Biases
    output_dir = "outputs",
)

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        custom_reward_function,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

### Task: Negative Sentiment with ClapAI model (Binary reward)

#### Load Model and Add LoRA

In [ ]:
from unsloth import FastModel
import torch

max_seq_length = 1024

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 64,           
    lora_alpha = 64,  
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

#### Data Preparation

In [ ]:
import re
from datasets import load_dataset, Dataset


SYSTEM_PROMPT = f"""
You are a helpful and conversational assistant. Always respond with a clearly negative sentiment.
"""

# load dataset function
def get_instructions() -> Dataset:
    # Load the dataset
    dataset_id = "dmitva/human_ai_generated_text"
    raw_dataset = load_dataset(dataset_id, split="train[:10%]")
    data = raw_dataset.map(lambda x: {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['instructions'].split("Task: ")[1]}
        ]
    },
    remove_columns=['id', 'human_text', 'ai_text', 'instructions']
    )
    return data

dataset = get_instructions()

#### Load ClapAI model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_id = "clapAI/modernBERT-base-multilingual-sentiment"
# Load the tokenizer and model
tokenizer_sent = AutoTokenizer.from_pretrained(model_id)
model_sent = AutoModelForSequenceClassification.from_pretrained(model_id, torch_dtype=torch.float16)

model_sent.to("cuda:1")
model_sent.eval()

# Retrieve labels from the model's configuration
id2label = model_sent.config.id2label

#### Reward functions

In [ ]:
# Custom reward function for GRPOTrainer
def custom_reward_function(prompts, completions, **kwargs):
    rewards = []
    responses = [completion[0]["content"] for completion in completions]
    print("="*50)
    last_prompt = ""
    for prompt, response in zip(prompts, responses):

        length = len(response)

        # Length thresholds
        min_len = 10     # penalize if shorter
        max_len = 2000    # penalize if longer

        if length < min_len or length > max_len:
            reward = 0
        else:
        
            inputs = tokenizer_sent(response, return_tensors="pt").to("cuda:1")
            with torch.inference_mode():
                outputs = model_sent(**inputs)
                predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
                neg = predictions[0][0]
            if neg > 0.6:
                reward = 1
            else:
                reward = 0
        rewards.append(reward)
        if last_prompt != prompt:
            print(f"Prompt:\n{prompt[1]['content']}")
            print("*"*20)
        print(f"Response:\n{response[:200]}{'...' if len(response) > 200 else ''} Reward: {reward}")
        print("-"*20)
        last_prompt = prompt

    return rewards

#### Model training

In [ ]:
from trl import GRPOConfig, GRPOTrainer
from unsloth import is_bfloat16_supported

max_prompt_length = 256

training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 1, 
    num_generations = 2,
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    max_steps = 200,
    save_steps = 200,
    max_grad_norm = 0.1,
    report_to = "wandb", # Can use Weights & Biases
    output_dir = "outputs",
)

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        custom_reward_function,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

## Saving fine-tuned model

In [ ]:
model.push_to_hub("HF_Username/model_name", token = "hf_token")
tokenizer.push_to_hub("HF_Username/model_name", token = "hf_token")

## Inference

In [ ]:
from unsloth import FastModel
import torch
max_seq_length = 1024

model, tokenizer = FastModel.from_pretrained(
    model_name = "your_saved_model", # Your model name
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

In [ ]:
# change depending on the task you want to do
reasoning_start = "<thinking>"
reasoning_end   = "</thinking>"
response_start = "<neg_sent_response>"
response_end = "</neg_sent_response>"

SYSTEM_PROMPT = f"""
You are a helpful and conversational assistant.
When responding, first analyze the user's instruction carefully and think through your reasoning for why a negative sentiment is appropriate.
Enclose your reasoning between the tags {reasoning_start} and {reasoning_end}.
Afterward, provide your response with a clearly negative sentiment, enclosed between the tags {response_start} and {response_end}.
Always follow this format and ensure your response reflects negative sentiment.
"""

In [ ]:
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user",   "content": """ 
    Hello, how are you doing today?
    """}
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    tokenize = False,
)

In [ ]:
from transformers import TextStreamer
output = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 768, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 0.1, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)